<a href="https://colab.research.google.com/github/dijkstra001/py-for-data/blob/main/py_for_data_case05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Utilização de Python para Análise de Dados: Caso prático - 05**

---

O conteúdo apresentado a seguir tem como objetivo criar cenários práticos e de uso geral onde a linguagem Python pode nos auxiliar de forma rápida e simples na análise de dados.

Esse material tem como objetivo ensinar algumas práticas para facilitar a análise de dados, não focando em técnicas complexas e de uso exclusivo para determinada situação.

O conteúdo tem como base o Livro **Python para Análise de Dados** (Wes McKinney, NOVATEC) e **Data Science do Zero** (Joel Grus, O'Reilly).

### **Extraíndo dados da internet:**



In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests

### **Conjunto de dados:**

Vamos utilizar o conjunto de dados disponível no site: https://www.valor.srv.br/indices/inpc.php, onde vamos analisar os INPC ao longo do tempo.

Importante destacar que vamos realizar a coleta dos dados através de web scraping e é sempre indispensável analisar o arquivo robots.txt. Nele, vão estar descritos os parâmetros que precisamos respeitar na hora de realizar a busca, além, é claro, se é permitido realizar tal tarefa.

No site em questão, até o momento do desenvolvimento desse exemplo, não foi identificado esse arquivo sobre quaisquer restrições.
Vamos nos limitar a buscar apenas **1 página** de dados.

In [2]:
# Realizando a requisição:
request = requests.get('https://www.valor.srv.br/indices/inpc.php').text

# Transformando em um objeto BeautifulSoup:
data = BeautifulSoup(request, 'html.parser')

In [3]:
# Analisando as informações no site, os dados de interesse estão dentro de tags <td's>:

# Filtrando apenas os dados de INPC:
dataset_desnormalizado = data.find_all('td')

# Mostrando as 5 primeiras linhas:
dataset_desnormalizado[:5]

[<td align="center" alt="Maio de 2021" title="Maio de 2021">
 							Mai/2021						</td>,
 <td align="center" alt="INPC de Maio de 2021: 0,96%" title="INPC de Maio de 2021: 0,96%">
 							0,96						</td>,
 <td align="center" alt="INPC acumulada no ano de 2021 (até Maio de 2021): 3,3316%" title="INPC acumulada no ano de 2021 (até Maio de 2021): 3,3316%">
 							3,3316						</td>,
 <td align="center" alt="INPC acumulada nos últimos 12 meses: 8,8962%" title="INPC acumulada nos últimos 12 meses: 8,8962%">
 							8,8962						</td>,
 <td align="center" alt="Abril de 2021" title="Abril de 2021">
 							Abr/2021						</td>]

In [4]:
# Realizando a contrução do nosso dataset normalizado:
dados = []

for i in dataset_desnormalizado[:140]:
  dados.append(i.get_text())

copia = dados.copy()
dados.clear()

for i in copia:
  i = i.replace('\n', '')
  i = i.replace('\t', '')
  dados.append(i)

dataset = {}
indice = 0
data = 0
perc = 1
perc_acum_ano = 2
perc_acum_12_meses = 3

for i in dados:
  dataset[indice] = {'data': dados[data: data + 1], 
                     'perc': dados[perc: perc + 1], 
                     'perc_acum_ano': dados[perc_acum_ano: perc_acum_ano + 1], 
                     'perc_acum_12_meses': dados[perc_acum_12_meses: perc_acum_12_meses + 1]
                     }
  indice += 1
  data += 4
  perc += 4 
  perc_acum_ano += 4 
  perc_acum_12_meses += 4 

  if data + 1 >= len(dados):
    break

In [5]:
for k, v in dataset.items():
  print(v)

  if k > 10:
    break

{'data': ['Mai/2021'], 'perc': ['0,96'], 'perc_acum_ano': ['3,3316'], 'perc_acum_12_meses': ['8,8962']}
{'data': ['Abr/2021'], 'perc': ['0,38'], 'perc_acum_ano': ['2,3491'], 'perc_acum_12_meses': ['7,5911']}
{'data': ['Mar/2021'], 'perc': ['0,86'], 'perc_acum_ano': ['1,9616'], 'perc_acum_12_meses': ['6,9373']}
{'data': ['Fev/2021'], 'perc': ['0,82'], 'perc_acum_ano': ['1,0922'], 'perc_acum_12_meses': ['6,2163']}
{'data': ['Jan/2021'], 'perc': ['0,27'], 'perc_acum_ano': ['0,2700'], 'perc_acum_12_meses': ['5,5315']}
{'data': ['Dez/2020'], 'perc': ['1,46'], 'perc_acum_ano': ['5,4473'], 'perc_acum_12_meses': ['5,4473']}
{'data': ['Nov/2020'], 'perc': ['0,95'], 'perc_acum_ano': ['3,9299'], 'perc_acum_12_meses': ['5,1979']}
{'data': ['Out/2020'], 'perc': ['0,89'], 'perc_acum_ano': ['2,9519'], 'perc_acum_12_meses': ['4,7706']}
{'data': ['Set/2020'], 'perc': ['0,87'], 'perc_acum_ano': ['2,0437'], 'perc_acum_12_meses': ['3,8879']}
{'data': ['Ago/2020'], 'perc': ['0,36'], 'perc_acum_ano': ['1,16

Construído o dataset normalizado, precisamos analisar os dados para que possamos extrair informações relevantes. Podemos perceber já de início que os dados não estão no formato adequado e precisamos tratá-los.

### **Realizando o processo de análise dos dados**:###

